In [1]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
import wandb

For this homework, we will use the Boston Housing Dataset, that we split in three: training, validation, and test set.

In [17]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train)

As a reference, let's see what MSE we get using a dummy regressor. To get your point for this homework, you should do better than this reference.

In [18]:
dummy = DummyRegressor().fit(X_train,y_train)
mean_squared_error(dummy.predict(X_test),y_test)

87.90414741540626

It will make our life much simpler to wrap the dataset as a torch one because it was allow us to create a dataloader which will take care of batching. On top of this, it is super easy to do. Make sure to read this carefully.

In [7]:
class Dataset(Dataset):
    def __init__(self,X,y):
        self.X = torch.tensor(X,dtype=torch.float)
        self.y = torch.tensor(y,dtype=torch.float)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [8]:
train_dataset = Dataset(X_train,y_train)

In [9]:
batch_size=32

Here we go, now that we have a proper torch dataset, we can create a data loader for the training set. If the dataset was truly large, we should also create dataloaders for the validation and the test set.

In [19]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

**Problem 1**: design a neural network model. Your model should have at two hidden layers. You are free to choose the non-linearities. You might consider using dropout layers, and the Sequential container to simplify your model specification. If you want to try other types of layers, go for it! 

In [11]:
class Model(nn.Module):
    def __init__(self,size1,size2):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
          nn.Linear(X.shape[1],size1),      
          nn.Dropout(p=0.5),
          nn.ReLU(),
          nn.Linear(size1,size2),
          nn.Dropout(p=0.5),
          nn.ReLU(),
          nn.Linear(size2,1)
        )
        
    def forward(self,x):
        return self.layers(x)

In [12]:
model = Model(3,4)

**Problem 2**: Implement an algorithm to train your model. A few hints:<br>
<ul>
<li> This function will likely be over 30 lines of code, so plan ahead.
<li> First, you should create an optimizer. You can pick whichever you want. One of the most popular one is the Adam optimizer, but there are many others: https://pytorch.org/docs/stable/optim.html
<li> If you decide to use wandb, you should remember to initialize it before the iterations starts.
<li> Think ahead about how you are going to keep track of the train and validation loss!
<li> The rest of the code is one big loop that will iterate epochs times 
<li> The body of the loop has two main tasks:
<ul>
    <li> First, the actual training. This is where you should use your dataloader to do the training by batches. That means you will have a loop that does the usual training procedure. 
    <li> Second, you should compute the validation loss. For simplicity, you can do it using sklearn.
    <li> Finally, make sure to keep track of the current model if it turns out to work well on the validation set!  
</ul>
<li> It is very likely that training the model will take a while. Not only because you will need to figure out what are the right parameters (e.g. learning rate of your optimizer, rate of dropout, size of the hidden layers), but also because errors in your code may go unnoticed. 
</ul>

In [13]:
def train(epochs):
    
    wandb.init(project='GNN',entity='tristanj')
    
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
    
    best_mse = None
    best_model = None
    
    for epoch in tqdm(range(epochs)):
        
        model.train()
        
        running_train_loss = 0.
        
        for batch_x, batch_y in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_x).reshape(1,-1)
            mse = ((y_pred - batch_y)**2).sum()
            running_train_loss += mse.item()
            mse.backward()
            optimizer.step()
            
        running_train_loss /= len(train_dataset)       
            
        model.eval()
            
        y_pred = model(torch.tensor(X_val,dtype=torch.float)).detach().numpy()
        mse = mean_squared_error(y_pred,y_val)   
        if not best_mse:
            best_mse = mse
            torch.save(model, 'best-model.pt') 
        if mse < best_mse:
            best_mse = mse
            torch.save(model, 'best-model.pt') 
            
        wandb.log({"train loss": running_train_loss, "val loss": mse, "best": best_mse})

In [14]:
train(10)

wandb: Currently logged in as: tristanj (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/1000 [00:00<?, ?it/s]

**Problem 3**: Write a function that evaluates your model on the test set.

In [15]:
def test():
    model = torch.load('best-model.pt')
    y_pred = model(torch.tensor(X_test,dtype=torch.float)).detach().numpy()
    return mean_squared_error(y_pred,y_test)

In [16]:
test()

22.019421438952158